# first try

In [2]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

X_train = pd.read_csv('data/X_train.csv')
y_train = pd.read_csv('data/y_train.csv')
X_test = pd.read_csv('data/X_test.csv')



forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)


y_pred = pd.DataFrame(y_pred, columns=['prediction'])
y_pred['ID'] = y_pred.index
y_pred

c:\Users\Jan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,prediction,ID
0,0,0
1,1,1
2,1,2
3,0,3
4,1,4
...,...,...
1461,1,1461
1462,1,1462
1463,0,1463
1464,0,1464


In [4]:
# this file should be submitted
y_pred.to_csv('data/y_pred.csv', index=False)